In [1]:
import logging
from prepare_data import HCDALoader
from sklearn.model_selection import KFold

logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')

In [2]:
loader = HCDALoader()

# load index values from main table
app_ix = loader.applications_train_index()

2018-06-24 12:32:48,285 - DEBUG - Reading application_train.csv...
2018-06-24 12:32:55,452 - DEBUG - Finished reading application_train.csv


In [3]:
# fit model using k-fold verification
kf = KFold(n_splits=4, shuffle=True)
for fold_indexes in kf.split(app_ix):
    print(fold_indexes[0].shape, fold_indexes[1].shape)
    
data_train, target_train, data_val, target_val = loader.load_train_val(fold_indexes[0], fold_indexes[1])

(230633,) (76878,)
(230633,) (76878,)
(230633,) (76878,)
(230634,) (76877,)


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/scipy/linalg/basic.py:1226: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  warnings.warn(mesg, RuntimeWarning)


In [4]:
data_train.shape, target_train.shape, data_val.shape, target_val.shape

((230634, 238), (230634,), (76877, 238), (76877,))

In [5]:
#from imblearn.combine import SMOTEENN

#smote_enn = SMOTEENN()
#data_train_os, target_train_os = smote_enn.fit_sample(data_train, target_train)

In [6]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=0)
data_train_os, target_train_os = ros.fit_sample(data_train, target_train)

In [7]:
import numpy as np

np.unique(target_train_os, return_counts=True)

(array([0, 1]), array([212026, 212026]))

In [8]:
from keras.models import Sequential
from keras.layers import Dense

model = Sequential()

model.add(Dense(units=64, activation='relu', input_dim=data_train_os.shape[1]))
model.add(Dense(units=1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=['accuracy'])

Using TensorFlow backend.


In [9]:
model.fit(data_train_os, target_train_os, epochs=5, batch_size=32, validation_data=(data_val, target_val))

Train on 424052 samples, validate on 76877 samples
Epoch 1/5
424052/424052 [==============================] - 24s 56us/step - loss: 0.6014 - acc: 0.6775 - val_loss: 1.3231 - val_acc: 0.9179
Epoch 2/5
424052/424052 [==============================] - 23s 55us/step - loss: 0.5812 - acc: 0.6943 - val_loss: 4.2812 - val_acc: 0.7322
Epoch 3/5
424052/424052 [==============================] - 23s 54us/step - loss: 0.5715 - acc: 0.7021 - val_loss: 4.6390 - val_acc: 0.7097
Epoch 4/5
424052/424052 [==============================] - 23s 55us/step - loss: 0.5632 - acc: 0.7091 - val_loss: 2.4363 - val_acc: 0.8480
Epoch 5/5
424052/424052 [==============================] - 23s 55us/step - loss: 0.5550 - acc: 0.7158 - val_loss: 2.3401 - val_acc: 0.8541
